In [5]:
import tensorflow as tf
import keras_cv
import json
import os

In [6]:
# Load annotations from a COCO JSON file
def load_annotations(annotation_file):
    with open(annotation_file, 'r') as f:
        annotations = json.load(f)
    return annotations

In [7]:
# Load image and annotation data
def load_image_and_annotations(image_path, annotations):
    image = tf.io.decode_jpeg(tf.io.read_file(image_path), channels=3)
    image_info = [ann for ann in annotations['images'] if ann['file_name'] == os.path.basename(image_path)]
    if not image_info:
        raise ValueError(f"Image {image_path} not found in annotations")
    
    image_id = image_info[0]['id']
    bboxes = [ann['bbox'] for ann in annotations['annotations'] if ann['image_id'] == image_id]
    classes = [ann['category_id'] for ann in annotations['annotations'] if ann['image_id'] == image_id]

    return image, bboxes, classes


In [8]:
# Data loader function for a tf.data.Dataset
def data_loader(annotation_file, image_folder):
    annotations = load_annotations(annotation_file)
    
    image_files = [os.path.join(image_folder, img['file_name']) for img in annotations['images']]
    
    def load_data(image_file):
        image, bboxes, classes = load_image_and_annotations(image_file, annotations)
        # Normalize bounding boxes and convert classes to tensors
        bboxes = tf.convert_to_tensor(bboxes, dtype=tf.float32)
        classes = tf.convert_to_tensor(classes, dtype=tf.int32)
        
        # You may need additional preprocessing steps, like resizing or normalization
        return image, {'boxes': bboxes, 'classes': classes}
    
    # Create a TensorFlow dataset from the image files
    dataset = tf.data.Dataset.from_tensor_slices(image_files).map(load_data)
    
    return dataset

In [ ]:
# Data loader function for a tf.data.Dataset
def data_loader(annotation_file, image_folder):
    annotations = load_annotations(annotation_file)
    
    image_files = [os.path.join(image_folder, img['file_name']) for img in annotations['images']]
    
    def load_data(image_file):
        image, bboxes, classes = load_image_and_annotations(image_file, annotations)
        # Normalize bounding boxes and convert classes to tensors
        bboxes = tf.convert_to_tensor(bboxes, dtype=tf.float32)
        classes = tf.convert_to_tensor(classes, dtype=tf.int32)
        
        # You may need additional preprocessing steps, like resizing or normalization
        return image, {'boxes': bboxes, 'classes': classes}
    
    # Create a TensorFlow dataset from the image files
    dataset = tf.data.Dataset.from_tensor_slices(image_files).map(load_data)
    
    return dataset